In [73]:
import larch, numpy, pandas, os
from larch import P, X

In [74]:
data = pandas.read_csv("./data/swissmetro.dat", sep="\t")
condition = (data.PURPOSE.isin([1,3]) & (data.CHOICE != 0))
df = data.loc[condition].copy()
df.shape

(6768, 28)

In [75]:
df.columns

Index(['GROUP', 'SURVEY', 'SP', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO',
       'LUGGAGE', 'AGE', 'MALE', 'INCOME', 'GA', 'ORIGIN', 'DEST', 'TRAIN_AV',
       'CAR_AV', 'SM_AV', 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'SM_TT', 'SM_CO',
       'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO', 'CHOICE'],
      dtype='object')

In [76]:
# Scale travel time and travel cost
df[['TRAIN_TT', 'SM_TT', 'CAR_TT']] = df[['TRAIN_TT', 'SM_TT', 'CAR_TT']] / 100
df["Free_ticket"] = 1 - df.GA
df['TRAIN_CO'] = df['TRAIN_CO'] * df["Free_ticket"] / 100
df['SM_CO'] = df['SM_CO'] * df["Free_ticket"] / 100
df['CAR_CO'] = df['CAR_CO'] * df["Free_ticket"] / 100

In [77]:
# For clarity, define these mode
TRAIN = 1
SM = 2
CAR = 3

In [78]:
alt_codes = [TRAIN, SM, CAR]
alt_names = ["Train", "SwissMetro", "Car"]

dfs = larch.DataFrames(co=df, alt_codes=alt_codes, 
                       alt_names=alt_names, ch_name="CHOICE")

In [79]:
m = larch.Model(dataservice=dfs)
m.title = "Travel mode choice"

In [80]:
df.head().T

,0,1,2,3,4
GROUP,2.00,2.00,2.00,2.00,2.00
SURVEY,0.00,0.00,0.00,0.00,0.00
SP,1.00,1.00,1.00,1.00,1.00
ID,1.00,1.00,1.00,1.00,1.00
PURPOSE,1.00,1.00,1.00,1.00,1.00
FIRST,0.00,0.00,0.00,0.00,0.00
TICKET,1.00,1.00,1.00,1.00,1.00
WHO,1.00,1.00,1.00,1.00,1.00
LUGGAGE,0.00,0.00,0.00,0.00,0.00
AGE,3.00,3.00,3.00,3.00,3.00


In [81]:
m.utility_co[TRAIN] = (
    + P.ASC_TRAIN
    + P.TT * X.TRAIN_TT
    + P.CO * X.TRAIN_CO
)
m.utility_co[SM] = (
    + P.TT * X.SM_TT
    + P.CO * X.SM_CO
)
m.utility_co[CAR] = (
    + P.ASC_CAR
    + P.TT * X.CAR_TT
    + P.CO * X.CAR_CO
)

m.choice_co_code = "CHOICE"

m.availability_co_vars = {
    TRAIN: 'TRAIN_AV',
    SM: 'SM_AV',
    CAR: 'CAR_AV',
}

### Model Estimation

In [82]:
m.load_data()

In [83]:
m.dataframes.choice_avail_summary()

,name,chosen,available
1,Train,908.0,6768
2,SwissMetro,4090.0,6768
3,Car,1770.0,5607
< Total All Alternatives >,,6768.0,


In [84]:
result = m.maximize_loglike(method='slsqp')

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_CAR,-0.217924,0.0,0.0,-inf,inf,0,,-0.217924
ASC_TRAIN,-0.727504,0.0,0.0,-inf,inf,0,,-0.727504
CO,-1.063457,0.0,0.0,-inf,inf,0,,-1.063457
TT,-1.237383,0.0,0.0,-inf,inf,0,,-1.237383


In [85]:
m.parameter_summary()

Parameter,Value,Null Value
ASC_CAR,-0.2179,0.0
ASC_TRAIN,-0.7275,0.0
CO,-1.063,0.0
TT,-1.237,0.0


In [72]:
#### NOTE:
# The results seem different from Biogeme and Pylogit.
# This may need to be reinvestigated again !!!

In [86]:
 dir(P)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [87]:
P??